In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-8sz1f7zg7oy0ui"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

Could not load vllm class, check CUDA support and GPU RAM size


#### Temasek Foundation Dataset

In [19]:
# 1. Test cases (dataloading ...)
import sys 
sys.path.append("../notebook/")
from optm.soft_prompt import load_tf_data

train_data, test_data = load_tf_data("../data/processed_data_clean.json")

# reference metric fuction below ... we care about prediction of label, specifically "no"
# we care about precision more than we care abou recall on 'No' prediction 

# Node dataset should ideally be concise and pure application info included ...
from methods.meta_prompt import MetaPrompt, PromptMode

tf_meta_prompt = MetaPrompt(
    task = "Evaluate grant application, make a decision (Yes, No, Maybe) and a brief comment explanating your decision on why this project is likely to be accepted or rejected.",
    func_name = "evaluate_grant",
    inputs = ["project_description"],
    outputs = ["label", "comment"],
    input_types = ["str"],
    output_types = ["str", "str"],
    mode = PromptMode.PROMPT
)

# Prepare test cases :: input dict & output dict
test_cases = []
for prompt, label, comment in zip(train_data["prompt"], train_data["label"], train_data["comment"]):
    test_cases.append(({"project_description": prompt}, {"label": label, "comment": comment}))
    

# Specific Metric required for TF dataset (Worth refactor the code a bit)
from methods.evolnode import EvolNode 
node = EvolNode(tf_meta_prompt, None, None, get_response=get_endpoint_response, test_cases=test_cases,
                custom_metric_map=None) # setting manual test cases


In [24]:
output_dict = test_cases[0][1]
pred_dict = test_cases[1][1]




In [21]:
# Custom metric works on output dictionary, both element level and batch level (indiv score & overall score)

from typing import Union, Callable

def tf_name_to_metric(output_name: str) -> Callable: 
    """ 
    Dataset specific metric pointer
    """
    if output_name == "label":
        return tf_label_metric
    else:
        raise ValueError(f"No metric for output: {output_name}")

def tf_label_metric(target_outputs: Union[list, dict], pred_outputs: Union[list, dict],
                    beta: float = 5):
    """ 
    "No" prediction > "Yes" prediction 
    Recall > Precision
    Weighted score (recall + precision) with beta : 1 as weights 
    """
    
    if isinstance(target_outputs, dict):
        target_outputs = [target_outputs]
    if isinstance(pred_outputs, dict):
        pred_outputs = [pred_outputs]
        
    assert all("label" in target and "comment" in target for target in target_outputs), "Target outputs must contain 'label' and 'comment'"
    assert all("label" in pred and "comment" in pred for pred in pred_outputs), "Predicted outputs must contain 'label' and 'comment'"
    
    # Extract labels
    true_labels = [t["label"].lower().strip() for t in target_outputs]
    pred_labels = [p["label"].lower().strip() for p in pred_outputs]
    
    # Calculate metrics for "no" predictions
    true_no = sum(1 for label in true_labels if label == "no")
    pred_no = sum(1 for label in pred_labels if label == "no")
    true_pos_no = sum(1 for t, p in zip(true_labels, pred_labels) if t == "no" and p == "no")
    
    # Calculate precision and recall for "no"
    no_precision = true_pos_no / pred_no if pred_no > 0 else 0
    no_recall = true_pos_no / true_no if true_no > 0 else 0
    
    # Calculate precision and recall for "no"
    no_precision = true_pos_no / pred_no if pred_no > 0 else 0
    no_recall = true_pos_no / true_no if true_no > 0 else 0
    
    # Simple weighted score using beta
    beta = 5  # Keeping your beta parameter
    weighted_score = ((1 / (1 + beta)) * no_precision + (beta / (1 + beta)) * no_recall) if (no_precision + no_recall) > 0 else 0.0
    
    return weighted_score
    

tf_meta_prompt

In [2]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode
from methods.llm import get_groq_response, get_claude_response

# Code + Compilor Task
# mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]

node = EvolNode(mp, None, None, get_response=get_endpoint_response, test_cases=test_cases) # setting manual test cases

node.evolve("i1", replace=True, batch_size=20, num_runs=2, print_summary=True) # Scale up batch size


input_dict = {"name": "Dilireba"}
output_dict = node(input_dict, max_attempts=6) # Batch Inference with vLLM

# node.get_response = get_groq_response # fast sequential inference
# output_dict = node(input_dict, max_attempts=6, batch_inference=False)
 
print("Output dict: ", output_dict)

Processing LLM queries: 100%|██████████| 20/20 [00:10<00:00,  1.97it/s]


 :: Total time elapsed: 10.15s, 0 errors


Processing LLM queries: 100%|██████████| 29/29 [00:25<00:00,  1.13it/s]


 :: Total time elapsed: 25.63s, 0 errors


Processing LLM queries: 0it [00:00, ?it/s]


 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.33
  🎯 Functional fitness: 1.00
  ⭐ Global fitness:     0.67
  🔄 Compiled solutions:        17
  ⏱️ Time breakdown:
     :: Query time: 33.43s
     :: Evolution time: 10.19s
     :: Evaluation time: 55.46s
     :: Total time: 99.09s


📊 Code 0: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
name 're' is not defined
name 're' is not defined


📊 Code 1: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Failed to parse LLM response -- JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstLiteralError : 
invalid syntax (<unknown>, line 1)
Failed to parse LLM response -- JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstLiteralError : 
invalid syntax (<unknown>, line 1)


📊 Code 2: Fitness: 66.7%


📊 Code 3: Fitness: 66.7%


📊 Code 4: Fitness: 